In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

month_list = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
year = '2004'

# Create a dictionary for month name to number conversion
month_dict = {
    'january': '01', 'february': '02', 'march': '03', 'april': '04', 'may': '05', 'june': '06',
    'july': '07', 'august': '08', 'september': '09', 'october': '10', 'november': '11', 'december': '12'
}

# Open CSV file for writing
with open('dataset_' + year + '.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    # Write header
    csvwriter.writerow(['Date', 'Gold Price'])

    for month in month_list:
        # Get the webpage content
        url = f'https://www.goldpriceindia.com/gold-price-{month}-{year}.php'
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')

        # Extract the titles (dates)
        h4_titles = soup.find_all('h4')
        list_h4 = [title.text for title in h4_titles]

        # Extract the table data
        table_data = soup.find_all(class_='table-data dayyeartable')

        count = 0
        for td in table_data:
            th = td.find_all('th')
            tds = td.find_all('td')
            row_data = [td.text.strip() for td in tds]

            if len(row_data) == 4:
                continue

            # Extract date from the h4 titles
            date_str = str(list_h4[count])[14:]
            day = date_str[:2]
            year_str = date_str[-4:]
            month_value = month_dict[month]

            # Extract price, remove currency symbol and commas
            price_str = row_data[1].replace("₹", "").replace(",", "")
            price = int(price_str)

            # Write the data to CSV
            csvwriter.writerow([f'{day}-{month_value}-{year_str}', price])
            count += 1

        print(f'Processed data for {month} {year}')


Processed data for january 2004
Processed data for february 2004
Processed data for march 2004
Processed data for april 2004
Processed data for may 2004
Processed data for june 2004
Processed data for july 2004
Processed data for august 2004
Processed data for september 2004
Processed data for october 2004
Processed data for november 2004
Processed data for december 2004


In [ ]:
import pandas as pd

# Read the CSV files
df1 = pd.read_csv('/content/2008-2009.csv')
df2 = pd.read_csv('/content/2010-2024.csv')

# Concatenate the dataframes
combined_df = pd.concat([df1, df2])

# Save the combined dataframe to a new CSV file
combined_df.to_csv('gold_data.csv', index=False)


In [ ]:
df = pd.read_csv("/content/gold_data.csv")

In [ ]:
df

,Date,Gold Price
0,31-08-2008,1190
1,30-08-2008,1188
2,29-08-2008,1186
3,30-09-2008,1350
4,29-09-2008,1317
...,...,...
5649,05-05-2024,7068
5650,04-05-2024,7068
5651,03-05-2024,7039
5652,02-05-2024,7069


In [ ]:
import requests
import csv
from datetime import datetime, timedelta

def make_gapi_request(date):
    api_key = "goldapi-2karslwgmh9ln-io"
    symbol = "XAU"
    curr = "INR"

    url = f"https://www.goldapi.io/api/{symbol}/{curr}/{date}"

    headers = {
        "x-access-token": api_key,
        "Content-Type": "application/json"
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        result = response.json()
        if 'price' in result:
            gold_price = result['price']

            # Save data to CSV
            with open('gold_price.csv', mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([date, gold_price])

            return date, gold_price
        else:
            print(f"No price data available for {date}")
            return None, None
    except requests.exceptions.RequestException as e:
        print("Error:", str(e))
        return None, None

# Generate dates from 14-03-2024 to 21-05-2024
start_date = datetime(2024, 3, 14)
end_date = datetime(2024, 5, 21)
delta = timedelta(days=1)
date_range = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

# Iterate over the dates and make API requests
for date in date_range:
    formatted_date = date.strftime("%Y-%m-%d")
    result_date, result_price = make_gapi_request(formatted_date)
    if result_date is not None:
        print(f"Gold price on {result_date}: {result_price}")


Gold price on 2024-03-14: 179702.9727
Gold price on 2024-03-15: 179882.2319
No price data available for 2024-03-16
No price data available for 2024-03-17
Gold price on 2024-03-18: 178695.1472
Gold price on 2024-03-19: 178893.7756
Gold price on 2024-03-20: 179094.609
Gold price on 2024-03-21: 183821.4681
Gold price on 2024-03-22: 180719.2286
No price data available for 2024-03-23
No price data available for 2024-03-24
Gold price on 2024-03-25: 180947.9512
Gold price on 2024-03-26: 182694.7047
Gold price on 2024-03-27: 182760.2486
Gold price on 2024-03-28: 184062.854
Gold price on 2024-03-29: 184402.8783
No price data available for 2024-03-30
No price data available for 2024-03-31
No price data available for 2024-04-01
Gold price on 2024-04-02: 188806.0953
Gold price on 2024-04-03: 189433.6102
Gold price on 2024-04-04: 191307.9437
Gold price on 2024-04-05: 190584.5627
No price data available for 2024-04-06
No price data available for 2024-04-07
Gold price on 2024-04-08: 194648.1474
Gold 

In [ ]:
import requests
import csv
from datetime import datetime, timedelta

def make_gapi_request(date):
    api_key = "goldapi-3hja19lwitmjwg-io"
    symbol = "XAU"
    curr = "INR"

    url = f"https://www.goldapi.io/api/{symbol}/{curr}/{date}"

    headers = {
        "x-access-token": api_key,
        "Content-Type": "application/json"
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        result = response.json()
        if 'price_gram_24k' in result:
            gold_price_per_gram_24k = result['price_gram_24k']

            # Save data to CSV
            with open('gold_price.csv', mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([date, gold_price_per_gram_24k])

            return date, gold_price_per_gram_24k
        else:
            print(f"No price data available for {date}")
            return None, None
    except requests.exceptions.RequestException as e:
        print("Error:", str(e))
        return None, None

# Generate dates from 14-03-2024 to 21-05-2024
start_date = datetime(2024, 3, 14)
end_date = datetime(2024, 5, 21)
delta = timedelta(days=1)
date_range = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

# Iterate over the dates and make API requests
for date in date_range:
    formatted_date = date.strftime("%Y-%m-%d")
    result_date, result_price_per_gram_24k = make_gapi_request(formatted_date)
    if result_date is not None:
        print(f"Gold price per gram (24k) on {result_date}: {result_price_per_gram_24k}")


Gold price per gram (24k) on 2024-03-14: 5777.5847
Gold price per gram (24k) on 2024-03-15: 5783.3481
No price data available for 2024-03-16
No price data available for 2024-03-17
Gold price per gram (24k) on 2024-03-18: 5745.1824
Gold price per gram (24k) on 2024-03-19: 5751.5684
Gold price per gram (24k) on 2024-03-20: 5758.0254
Gold price per gram (24k) on 2024-03-21: 5909.9974
Gold price per gram (24k) on 2024-03-22: 5810.2581
No price data available for 2024-03-23
No price data available for 2024-03-24
Gold price per gram (24k) on 2024-03-25: 5817.6117
Gold price per gram (24k) on 2024-03-26: 5873.7712
Gold price per gram (24k) on 2024-03-27: 5875.8784
Gold price per gram (24k) on 2024-03-28: 5917.7582
Gold price per gram (24k) on 2024-03-29: 5928.6902
No price data available for 2024-03-30
No price data available for 2024-03-31
No price data available for 2024-04-01
Gold price per gram (24k) on 2024-04-02: 6070.2569
Gold price per gram (24k) on 2024-04-03: 6090.432
Gold price per